In [1]:
import nengo

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class RecAdapt(nengo.Network):

    def __init__(self, n_neurons: int, start_weights: np.ndarray,
                 learning_rate=1e-9, threshold=150, theta_tau=1.0,
                 sample_every=0.1, dt=0.001,
                 label=None, seed=None, add_to_container=None):
        super().__init__(label, seed, add_to_container)

        self.kappa = learning_rate * dt
        self.weights = start_weights.copy()
        self.n_nrns = n_neurons
        self.max_inhib = max_inhib

        self.dt = dt
        self.period = sample_every / dt
        self.weight_history = []
        self.lowpass = nengo.Lowpass(theta_tau).make_step(n_neurons, n_neurons, dt, None)

        self.in_rates = np.zeros(n_neurons)
        self.enabled = 0.

        self.in_neurons = nengo.Node(self.set_in_rates, size_in=n_neurons)
        self.out_neurons = nengo.Node(self.set_out_rates, size_in=n_neurons)
        self.enable = nengo.Node(self.set_enabled, size_in=1)
        self.output = nengo.Node(self.bcm_func, size_out=n_neurons)

    def set_in_rates(self, t, x):
        self.in_rates = x

    def set_out_rates(self, t, x):
        self.out_rates = x

    def set_enabled(self, t, x):
        self.enabled = x

    def bcm_func(self, t):
        theta = self.lowpass(t, self.out_rates)

        self.in_rates[self.in_rates < self.thresh] = 0.
        self.weights += np.outer(self.kappa * theta, self.in_rates) * self.enabled
        self.weights[self.weights < self.max_inhib] = self.max_inhib

        np.fill_diagonal(self.weights, 0)

        if ((t - self.dt) / self.dt % self.period) < 1:
            self.weight_history.append(self.weights.copy())

        return np.dot(self.weights, self.in_rates)

In [2]:
def stim_func(t):
    if t < 0.5:
        return 0.3
    elif 1.0 < t < 1.5:
        return 0.3
    else:
        return -0.3

In [3]:
ens_params = dict(encoders=[[1], [1], [-1], [-1]], intercepts=[-0.5, -0.1, -0.1, -0.5], max_rates=[250, 300, 300, 250])

In [ ]:
n_neurons = 4

with nengo.Network() as model:
    stim = nengo.Node(stim_func)

    ens = nengo.Ensemble(n_neurons, 1, neuron_type=nengo.LIFRate(), **ens_params)
    rec_learn = RecAdapt(n_neurons, rec_inhib, learning_rate=bcm_lr,
                         max_inhib=-0.05, sample_every=t_each / sample_freq)
    
    nengo.Connection(stim, ens)
    nengo.Connection(ens.neurons, rec_learn.in_neurons, synapse=0.01)
    nengo.Connection(rec_learn.output, ens.neurons, synapse=None)
    
    p_spikes = nengo.Probe(ens.neurons)

In [ ]:
with nengo.Simulator(model) as sim:
    sim.run(2)